<a href="https://colab.research.google.com/github/okechukwuchude/Automating-Medical-Coding/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import itertools
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
#ICD codes and meaningg
D_ICD_DIAG = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/D_ICD_DIAGNOSES.csv.gz',compression='gzip')
D_ICD_PROC = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/D_ICD_PROCEDURES.csv.gz',compression='gzip')

In [3]:
#original ICD files
diagnoses_icd = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/DIAGNOSES_ICD.csv.gz',compression='gzip')
procedures_icd = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/PROCEDURES_ICD.csv.gz',compression='gzip')

In [4]:
diagnoses_icd.head(5)

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559


In [5]:
diagnoses_icd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225345 entries, 0 to 225344
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   row_id      225345 non-null  int64  
 1   subject_id  225345 non-null  int64  
 2   hadm_id     225345 non-null  int64  
 3   seq_num     225339 non-null  float64
 4   icd9_code   225339 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 8.6+ MB


In [6]:
#Read notes source file
notes = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/NOTEEVENTS.csv.gz', compression='gzip')
keep = notes[['hadm_id','category','text']]

In [7]:
keep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880107 entries, 0 to 880106
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   hadm_id   861038 non-null  float64
 1   category  880107 non-null  object 
 2   text      880107 non-null  object 
dtypes: float64(1), object(2)
memory usage: 20.1+ MB


In [8]:
len(diagnoses_icd['icd9_code'].unique())


5054

In [9]:
keep

,hadm_id,category,text
0,NaN,Radiology,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,NaN,Radiology,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,NaN,Radiology,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,NaN,Radiology,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,NaN,Radiology,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...
880102,104049.0,Nursing/other,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,104049.0,Nursing/other,I have examined pt. & agree w/ [**First Name8 ...
880104,104049.0,Nursing/other,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,104049.0,Nursing/other,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [10]:
len(diagnoses_icd['hadm_id'].unique())

26836

In [11]:
#checking for null values
keep.isnull().sum()

hadm_id     19069
category        0
text            0
dtype: int64

In [12]:
# Dictionary to store diagnoses data with admission IDs as keys and lists of diagnosis ICD codes as values
diagnoses_dict = {}

# Iterate through each row in the diagnoses_icd DataFrame
for i in range(len(diagnoses_icd)):
    # Get the current row
    entry = diagnoses_icd.iloc[i]
    # Extract the admission ID and ICD code from the current row
    hadm = entry['hadm_id']
    icd = entry['icd9_code']
    # Check if the admission ID already exists in the diagnoses_dict
    if hadm not in diagnoses_dict:
        # If not, create a new entry with the admission ID as key and a list containing the ICD code as value
        diagnoses_dict[hadm] = [icd]
    else:
        # If the admission ID already exists, append the ICD code to the existing list of codes
        diagnoses_dict[hadm].append(icd)


In [13]:
# Dictionary to store procedures data with admission IDs as keys and lists of procedure ICD codes as values
procedures_dict = {}

# Iterate through each row in the procedures_icd DataFrame
for i in range(len(procedures_icd)):
    # Get the current row
    entry = procedures_icd.iloc[i]
    # Extract the admission ID and ICD code from the current row
    hadm = entry['hadm_id']
    icd = entry['icd9_code']
    # Check if the admission ID already exists in the procedures_dict
    if hadm not in procedures_dict:
        # If not, create a new entry with the admission ID as key and a list containing the ICD code as value
        procedures_dict[hadm] = [icd]
    else:
        # If the admission ID already exists, append the ICD code to the existing list of codes
        procedures_dict[hadm].append(icd)

In [14]:
diagnoses_df = pd.DataFrame.from_dict(diagnoses_dict,orient='index')
procedures_df = pd.DataFrame.from_dict(procedures_dict,orient='index')

In [24]:
diagnoses_df

,DIAG_CODE1,DIAG_CODE2,DIAG_CODE3,DIAG_CODE4,DIAG_CODE5,DIAG_CODE6,DIAG_CODE7,DIAG_CODE8,DIAG_CODE9,DIAG_CODE10,...,DIAG_CODE31,DIAG_CODE32,DIAG_CODE33,DIAG_CODE34,DIAG_CODE35,DIAG_CODE36,DIAG_CODE37,DIAG_CODE38,DIAG_CODE39,DIAG_CODES
HADM_ID,,,,,,,,,,,,,,,,,,,,,
163353,V3001,V053,V290,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3001,V053,V290"
145834,0389,78559,5849,4275,41071,4280,6826,4254,2639,None,...,None,None,None,None,None,None,None,None,None,"0389,78559,5849,4275,41071,4280,6826,4254,2639"
178980,V3000,V053,V290,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3000,V053,V290"
118037,V3001,V053,V290,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3001,V053,V290"
159514,V3001,7706,7746,V290,V502,V053,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3001,7706,7746,V290,V502,V053"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128876,V3000,V290,V053,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3000,V290,V053"
105824,V3001,76502,7705,7702,769,7742,7793,7756,7470,77082,...,None,None,None,None,None,None,None,None,None,"V3001,76502,7705,7702,769,7742,7793,7756,7470,..."
104049,V3000,7731,76517,76527,V290,V053,V502,None,None,None,...,None,None,None,None,None,None,None,None,None,"V3000,7731,76517,76527,V290,V053,V502"


In [25]:
# Rename the columns of the diagnoses DataFrame to include a prefix 'DIAG_CODE' followed by a number for each column
diagnoses_df.columns = ['DIAG_CODE'+str(i) for i in range(1, len(diagnoses_df.columns) + 1)]

# Set the name of the index of the diagnoses DataFrame to 'HADM_ID'
diagnoses_df.index.name = 'hadm_id'

# Rename the columns of the procedures DataFrame to include a prefix 'PRCD_CODE' followed by a number for each column
procedures_df.columns = ['PRCD_CODE'+str(i) for i in range(1, len(procedures_df.columns) + 1)]

# Set the name of the index of the procedures DataFrame to 'HADM_ID'
procedures_df.index.name = 'hadm_id'

# Merge the diagnoses and procedures DataFrames using an outer join based on the 'HADM_ID' column
# The resulting DataFrame will contain all unique 'HADM_ID' values from both DataFrames,
# with diagnosis and procedure codes aligned accordingly. Missing values will be filled with NaN.
codes_df = pd.merge(diagnoses_df, procedures_df, how='outer', on='hadm_id')


In [26]:
# For each row in the diagnoses DataFrame, join all non-null entries (diagnosis codes) into a single string, separated by commas
diagnoses_df['DIAG_CODES'] = diagnoses_df[diagnoses_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

# For each row in the procedures DataFrame, join all non-null entries (procedure codes) into a single string, separated by commas
procedures_df['PROC_CODES'] = procedures_df[procedures_df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)


In [27]:
# Extract the 'DIAG_CODES' column from the diagnoses DataFrame
diagnoses = diagnoses_df[['DIAG_CODES']]

# Extract the 'PROC_CODES' column from the procedures DataFrame
procedures = procedures_df[['PROC_CODES']]

# Merge the 'DIAG_CODES' and 'PROC_CODES' DataFrames using an outer join based on the 'HADM_ID' column
codes = pd.merge(diagnoses, procedures, how='outer', on='hadm_id')

# Drop any rows with missing values (NaN) from the merged DataFrame
codes = codes.dropna()


In [28]:
codes.to_csv('CODES.csv')

In [29]:
print(keep.columns)

Index(['hadm_id', 'category', 'text'], dtype='object')


In [30]:
print(codes.columns)

Index(['DIAG_CODES', 'PROC_CODES'], dtype='object')


In [38]:
# Merge the 'KEEP' DataFrame with the 'codes' DataFrame using a left join based on the 'HADM_ID' column
merged_df = pd.merge(keep, codes, how='left', on='hadm_id')

# Drop any rows with missing values (NaN) from the merged DataFrame
merged_df = merged_df.dropna()
merged_df= merged_df.set_index('hadm_id')


In [39]:
merged_df

,category,text,DIAG_CODES,PROC_CODES
hadm_id,,,,
102314.0,Radiology,"[**2119-2-3**] 4:10 PM\n US ABD LIMIT, SINGLE ...","5722,07054,5715,5724,5185,5070,99859,4538,5849...","5059,311.0,3324.0,9672.0,5011.0,3893.0,3995.0,..."
102314.0,Radiology,"[**2119-2-4**] 10:38 AM\n US ABD LIMIT, SINGLE...","5722,07054,5715,5724,5185,5070,99859,4538,5849...","5059,311.0,3324.0,9672.0,5011.0,3893.0,3995.0,..."
185945.0,Radiology,[**2167-2-14**] 10:41 AM\n CHEST (PA & LAT) ...,"4240,4280,51881,4254,42611,9971,42731,4239,998...","3524,3403.0,3409.0,3961.0,3723.0,8856.0,8853.0..."
185945.0,Radiology,[**2167-2-16**] 7:50 PM\n CHEST (PORTABLE AP) ...,"4240,4280,51881,4254,42611,9971,42731,4239,998...","3524,3403.0,3409.0,3961.0,3723.0,8856.0,8853.0..."
185945.0,Radiology,[**2167-2-17**] 2:24 AM\n CHEST (PORTABLE AP) ...,"4240,4280,51881,4254,42611,9971,42731,4239,998...","3524,3403.0,3409.0,3961.0,3723.0,8856.0,8853.0..."
...,...,...,...,...
104049.0,Nursing/other,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...,"V3000,7731,76517,76527,V290,V053,V502,V3000,77...","640,9983.0,9955.0,640.0,9983.0,9955.0"
104049.0,Nursing/other,I have examined pt. & agree w/ [**First Name8 ...,"V3000,7731,76517,76527,V290,V053,V502,V3000,77...","640,9983.0,9955.0,640.0,9983.0,9955.0"
104049.0,Nursing/other,NPN nights\n\n\nFluids/Nutrition: Weight 2025...,"V3000,7731,76517,76527,V290,V053,V502,V3000,77...","640,9983.0,9955.0,640.0,9983.0,9955.0"


In [40]:
sample = merged_df.sample(n=20000)


In [41]:
sample.to_csv('sample_20k.csv')


In [43]:
sample.columns

Index(['category', 'text', 'DIAG_CODES', 'PROC_CODES'], dtype='object')

In [42]:
sample

,category,text,DIAG_CODES,PROC_CODES
hadm_id,,,,
119180.0,Nursing/other,"Neonatology Attending\n\nNow day of life 117, ...","77989,47879,7766,7793,7707,2511,36221,5583,530...","9671,9915.0,966.0,9904.0,9671.0,9915.0,966.0,9..."
118173.0,Nursing/other,"Nursing Note\n\n\n1. O/A Temps stable, continu...","V3001,76527,77089,76518,7788,V053,V290,V3001,7...","9390,9915.0,9955.0,9390.0,9915.0,9955.0"
139010.0,Nursing/other,CCU NURSING PROGRESS NOTE\nS:I JUST WANT TO SL...,"3962,5990,4019,5939,41401,2449,73300,3962,5990...","3723,8853.0,8856.0,9920.0,9904.0,3723.0,8853.0..."
113003.0,Nursing/other,Neonatology-[** 7**] PRogress Note\n\nPE: [**K...,"V3101,76503,7742,769,76523,77081,4589,75261,V2...","9604,9915.0,9983.0,331.0,3893.0,9904.0,9955.0,..."
131421.0,Nursing/other,Respiratory Care\nPt remains trached cuff infl...,"51883,5119,42832,99662,40391,42731,99661,70703...","3491,3491.0,8872.0,3893.0,3895.0,3995.0,966.0,..."
...,...,...,...,...
108001.0,Nursing/other,Respiratory Care Note\nPt given aerosol nebs w...,"1970,1961,99811,42731,4251,3970,4280,E8786,V10...","324,4029.0,3422.0,3403.0,3322.0,9904.0,324.0,4..."
124650.0,Nursing/other,csru adm\npt adm initially under effects of an...,"41401,4111,42741,42731,4275,4271,5990,48241,99...","3615,3722.0,3613.0,3961.0,8853.0,8855.0,3791.0..."
194773.0,Nursing/other,"RESPIRATORY CARE:\n\nPt remains trached, minim...","4210,4275,51884,03811,4821,5845,4411,70703,112...","3965,3511.0,3712.0,3521.0,3845.0,3845.0,7761.0..."


**Codes To Dictionary**

In [44]:
sample_ids = sample.index


In [45]:
flt_diag = diagnoses_icd[diagnoses_icd['hadm_id'].isin(sample_ids)]
flt_proc = procedures_icd[procedures_icd['hadm_id'].isin(sample_ids)]